<table align="left"><td>
  <a target="_blank"  href="https://github.com/Dhruv0208/Research-Project/Autoencoder.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on github
  </a>
</td><td>
  <a target="_blank"  href="https://colab.sandbox.google.com/github/Dhruv0208/Research-Project/Autoencoder.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
</td></table>

In [1]:
# import sys
# !{sys.executable} -m pip install keras
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install imutils
# !{sys.executable} -m pip install opencv-python
# !{sys.executable} -m pip install sklearn
# !{sys.executable} -m pip install --upgrade pip
# !{sys.executable} -m pip install -q tfds-nightly tensorflow
# !{sys.executable} -m pip install pydot
# !{sys.executable} -m pip install graphviz
# !{sys.executable} -m pip install pydotplus

In [2]:
import keras
from tensorflow.keras import layers
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.layers import Input
from tensorflow.python.keras.utils.vis_utils import plot_model
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import json
from functools import partial
import os
from imutils import paths
import cv2
import IPython.display as display
import multiprocessing

In [3]:
print(tf.__version__)

2.3.1


In [4]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}',format(device_name))

Found GPU at: {} /device:GPU:0


In [5]:
# splits = ['train', 'validation', 'test']
# (train_dataset, val_dataset, test_dataset), dataset_info = tfds.load(name='coco/2017', data_dir = '../dhruv', 
#                                                        split = list(splits), 
#                                                        download=False, with_info=True,
#                                                                     as_supervised=False)
# assert isinstance(train_dataset, tf.data.Dataset)
# assert isinstance(val_dataset, tf.data.Dataset)
# assert isinstance(test_dataset, tf.data.Dataset)
# print(train_dataset)
# print(val_dataset)
# print(test_dataset)

In [6]:
# file_pattern = f'coco/2017/1.1.0/coco-train.tfrecord*'
# files = tf.data.Dataset.list_files(file_pattern)

In [7]:
# len(files)

In [8]:
# train_ds = files.interleave(tf.data.TFRecordDataset,
#                            cycle_length = 4,
#                            num_parallel_calls = tf.data.experimental.AUTOTUNE)
# # train_ds = tf.data.TFRecordDataset(file_pattern)

In [9]:
# def read_tfrecord(serialized_example):
#     feature_description={
#         "image": tf.io.FixedLenFeature((), tf.string, "")
#     }
#     example = tf.io.parse_single_example(
#     serialized_example, feature_description
#     )
#     image = tf.io.decode_jpeg(example['image'], channels=3)
#     image = tf.image.convert_image_dtype(image, dtype=tf.float32)
#     image = image / 255
#     image = tf.image.resize(image, (512, 512))
#     return image

In [10]:
# cores = multiprocessing.cpu_count()
# print(cores)

# train_ds = train_ds.map(read_tfrecord, num_parallel_calls=cores)
# train_ds = train_ds.cache()

In [11]:
# for inputs in train_ds.take(1):
#     print(inputs)

In [12]:
# train_ds = train_ds.shuffle(1024).batch(1)
# train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)

In [13]:
tf.config.run_functions_eagerly(True)
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

In [14]:
strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.ReductionToOneDevice(),
                                          devices=["/gpu:0", "gpu:1", "/gpu:2", "/gpu:3"])
AUTOTUNE = tf.data.experimental.AUTOTUNE
IMAGE_SIZE = [512,512]
BATCH_SIZE_PER_REPLICA = 16
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


In [15]:
file_pattern_train = f'coco/2017/1.1.0/coco-train.tfrecord*'
file_pattern_test = f'coco/2017/1.1.0/coco-test.tfrecord*'
file_pattern_valid = f'coco/2017/1.1.0/coco-validation.tfrecord*'
Training_filenames = tf.data.Dataset.list_files(file_pattern_train)
Test_filenames = tf.data.Dataset.list_files(file_pattern_test)
Valid_filenames = tf.data.Dataset.list_files(file_pattern_valid)
print("Train Tfrecords Files: ", len(Training_filenames))
print("Test Tfrecords Files: ", len(Test_filenames))
print("Valid Tfrecords Files: ", len(Valid_filenames))

Train Tfrecords Files:  256
Test Tfrecords Files:  64
Valid Tfrecords Files:  8


In [16]:
def decode_image(image):
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, [*IMAGE_SIZE])
    return image

In [17]:
def read_tfrecord(serialized_example):
    feature_description={
        "image": tf.io.FixedLenFeature([], tf.string,)
    }
    example = tf.io.parse_single_example(
    serialized_example, feature_description
    )
    image = decode_image(example['image'])
    return image

In [18]:
def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(
        filenames)
    dataset = dataset.with_options(ignore_order)
    dataset =dataset.map(
        partial(read_tfrecord), num_parallel_calls=AUTOTUNE)
    return dataset

In [19]:
def get_dataset(filenames):
    dataset = load_dataset(filenames)
    dataset = dataset.shuffle(1024)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.batch(16)
    return dataset

In [20]:
train_dataset = get_dataset(Training_filenames)
valid_dataset = get_dataset(Valid_filenames)
test_dataset = get_dataset(Test_filenames)
train_dist_dataset = strategy.experimental_distribute_dataset(train_dataset)
valid_dist_dataset = strategy.experimental_distribute_dataset(valid_dataset)
test_dist_dataset = strategy.experimental_distribute_dataset(test_dataset)
# image_batch = next(iter(train_dataset))
# def show_batch(image_batch):
#     plt.figure(figsize=(10,10))
#     for n in range(25):
#         ax = plt.subplot(5,5, n+1)
#         print(image_batch[n].shape)
#         plt.imshow(image_batch[n] / 255.0)
#         plt.axis("off")
        
# show_batch(image_batch.numpy())

/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3356: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  warnings.warn(


## Preparing autoencoder

In [21]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 4


In [22]:
def get_compiled_model():
    input_img = Input(shape=(512,512,3))
    x = layers.Conv2D(256, (1,1), activation='relu', padding='same')(input_img)
    x = layers.MaxPooling2D((2,2), padding = 'same')(x)
    x = layers.Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,2), padding = 'same')(x)
    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,2), padding = 'same')(x)
    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,2), padding = 'same')(x)
    x = layers.Conv2D(16, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,2), padding='same')(x)
    x = layers.Conv2D(16, (3,3), activation='relu', padding='same')(x)
    encoded = layers.MaxPooling2D((2,2), padding='same')(x)
    x = layers.Conv2D(16, (3,3), activation='relu', padding='same')(encoded)
    x = layers.UpSampling2D((2,2))(x)
    x = layers.Conv2D(16, (3,3), activation = 'relu', padding = 'same')(x)
    x = layers.UpSampling2D((2,2))(x)
    x = layers.Conv2D(32, (3,3), activation = 'relu', padding= 'same')(x)
    x = layers.UpSampling2D((2,2))(x)
    x = layers.Conv2D(64, (3,3), activation = 'relu', padding= 'same')(x)
    x = layers.UpSampling2D((2,2))(x)
    x = layers.Conv2D(128, (3,3), activation = 'relu', padding= 'same')(x)
    x = layers.UpSampling2D((2,2))(x)
    x = layers.Conv2D(256, (3,3), activation = 'relu', padding= 'same')(x)
    x = layers.UpSampling2D((2,2))(x)
    decoded = layers.Conv2D(3, (3,3), activation='sigmoid', padding='same')(x)
    autoencoder = tf.keras.Model(input_img, decoded)
    return autoencoder

In [23]:
# plot_model(autoencoder, show_shapes=True)

## Extracting Features from intermediate layers

In [24]:
# intermediate_layer_model_1 = keras.Model(inputs=autoencoder.inputs,
#                                       outputs=autoencoder.get_layer('max_pooling2d_1').output)
# features_1 = intermediate_layer_model_1(train_ds)
# intermediate_layer_model_2 = keras.Model(inputs=autoencoder.inputs,
#                                       outputs=autoencoder.get_layer('max_pooling2d_3').output)
# features_2 = intermediate_layer_model_1(train_ds)
# intermediate_layer_model_3 = keras.Model(inputs=autoencoder.inputs,
#                                       outputs=autoencoder.get_layer('max_pooling2d_5').output)
# features_3 = intermediate_layer_model_1(train_ds)

## Training and testing the model

In [25]:
# tcardinality = train_ds.cardinality()

In [26]:
# print((tcardinality == tf.data.UNKNOWN_CARDINALITY).numpy())

In [27]:
# print(tf.data.experimental.cardinality(train_ds).numpy())

In [28]:
## the code below is working don't delete
# for batch in train_ds:
#     autoencoder.fit(batch, batch,
#                    epochs=5)

In [29]:
with strategy.scope():
    model = get_compiled_model()
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    checkpoint = tf.train.Checkpoint(optimizer=optimizer, model = model)

In [30]:
with strategy.scope():
    loss_object = tf.keras.losses.CategoricalCrossentropy(
        from_logits=True,
        reduction=tf.keras.losses.Reduction.NONE)
    def compute_loss(images, predictions):
        per_example_loss = loss_object(images, predictions)
        return tf.nn.compute_average_loss(per_example_loss, global_batch_size=GLOBAL_BATCH_SIZE)

In [31]:
with strategy.scope():
    test_loss = tf.keras.metrics.Mean(name='test_loss')

    train_accuracy = tf.keras.metrics.CategoricalAccuracy(
      name='train_accuracy')
    test_accuracy = tf.keras.metrics.CategoricalAccuracy(
      name='test_accuracy')


In [32]:
def train_step(inputs):
    images = inputs
    
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = compute_loss(images, predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_accuracy.update_state(images, predictions)
    return loss

def test_step(inputs):
    images = inputs
    predictions = model(images, training=False)
    t_loss = loss_object(images, predictions)
    
    test_loss.update_state(t_loss)
    test_accuracy.update_state(images, predictions)

In [33]:
EPOCHS = 2

@tf.function
def distributed_train_step(dataset_inputs):
    per_replica_losses = strategy.run(train_step, args=(dataset_inputs,))
    return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses,
                         axis=None)

@tf.function
def distributed_test_step(dataset_inputs):
    return strategy.run(test_step, args=(dataset_inputs,))


for epoch in range(EPOCHS):
  # TRAIN LOOP
    total_loss = 0.0
    num_batches = 0
    for x in train_dist_dataset:
        total_loss += distributed_train_step(x)
        num_batches += 1
        train_loss = total_loss / num_batches

  # TEST LOOP
    for x in test_dist_dataset:
        distributed_test_step(x)

    if epoch % 2 == 0:
        checkpoint.save(checkpoint_prefix)

    template = ("Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, "
              "Test Accuracy: {}")
    print (template.format(epoch+1, train_loss,
                         train_accuracy.result()*100, test_loss.result(),
                         test_accuracy.result()*100))

    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()

## the point at which I have interrupted the code it does not move further than this. 
## it seem to like it gets stuck or deadlock.

Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:

KeyboardInterrupt: 

## Comparing test and model created images

In [ ]:
# n = 2
# plt.figure(figsize=(32,32))
# for i in range(n):
#     # display original
#     ax = plt.subplot(3, n, i+1)
#     plt.imshow(test_images[i])
#     plt.gray()
#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)

    
#     # display reconstruction
#     ax = plt.subplot(3, n, i+n+1)
#     plt.imshow(decoded_imgs[i])
#     plt.gray()
#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)
    
# plt.show()

## Visualizing loss

In [ ]:
# print(history.history.keys())

# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper right')
# plt.show()